In [ ]:
import sys
lib_path = [r'C:\Users\ikahbasi\OneDrive\Applications\GitHub\SeisRoutine',
            r'C:\Users\ikahb\OneDrive\Applications\GitHub\SeisRoutine']
for path in lib_path:
    sys.path.append(path)
##########################################################################
import SeisRoutine.catalog as src
import SeisRoutine.waveform as srw
import SeisRoutine.config as srconf
import SeisRoutine.statistics as srs

In [ ]:
import seisbench.data as sbd
import seisbench.generate as sbg
import seisbench.models as sbm
from seisbench.util import worker_seeding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import logging
from datetime import datetime
import os
import ipynbname
from IPython.display import display, HTML
from SeisRoutine.waveform.waveform import fft
from scipy import signal
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import obspy
import obspy.signal
from obspy.signal.cross_correlation import correlate
import numpy as np
import scipy.signal as scsig
import matplotlib.pyplot as plt
import numpy as np
import seisbench.models as sbm



In [ ]:
def corr (a, b):
    """
    Row-wise (vectorized):
    """
    a_m = a - np.mean(a, axis=1, keepdims=True)
    b_m = b - np.mean(b, axis=1, keepdims=True)

    cov = np.sum(a_m * b_m, axis=1)
    std = np.std(a, axis=1) * np.std(b, axis=1) * a.shape[1]

    row_corr_vec = cov / std
    return row_corr_vec

In [ ]:
def visual_eval(data_X, data_Y, models, phases='NPS', title=None):
    results = {}
    for key, model in models.items():
        with torch.no_grad():
            data = torch.tensor(data_X, device=model.device).unsqueeze(0)
            pred = model(data)
            pred = pred[0].cpu().numpy()
        results[key] = pred
    #
    nrows = len(models) + 2
    gridspec_kw = {"hspace": 0,
                   "height_ratios": [3] + [1]*(nrows-1)}
    #
    fig, axes = plt.subplots(
        nrows, 1,
        figsize=(10, 6),
        sharex=True,
        gridspec_kw=gridspec_kw)
    axes[0].plot(data_X.T + np.array([0, 1, 2]))
    axes[0].set_ylabel('Waveform')
    axes[0].set_yticks([0, 0.5, 1])
    #
    axes[1].plot(data_Y.T)
    axes[1].set_ylabel('Manual')
    nrow = 2
    for key, val in results.items():
        axes[nrow].plot(val.T, label=[ch for ch in phases])
        axes[nrow].set_ylabel(key)
        axes[nrow].set_yticks([0.5, 1])
        axes[nrow].legend()
        cc = corr(a=val, b=data_Y)
        axes[nrow].set_title(str(cc))
        nrow += 1
    fig.suptitle(title)


In [ ]:
init_cfg = srconf.load_config('0-init-cfg.yml')
cfg_path = os.path.join(init_cfg.target_config_filepath,
                        init_cfg.target_config_filename)
cfg = srconf.load_config(cfg_path)

In [ ]:
dataset = sbd.WaveformDataset(
    path=cfg.dataset.path,
    sampling_rate=cfg.training.dataset.sampling_rate,
    component_order=cfg.training.dataset.component_order,
          )

In [ ]:
# ii = 3
# step = 100
# for ii in range(ii*step, (ii+1)*step):
#     sample = dataset.get_sample(ii)
#     _data, _meta = sample
#     plt.plot(_data.T)
#     plt.legend(['E', 'N', 'Z'])
#     plt.title(f'{_meta['index']} {_meta['station_code']}')
#     plt.show()
#     # break

In [ ]:
phase_dict = {
    "trace_p_arrival_sample": "P",
    "trace_pP_arrival_sample": "P",
    "trace_P_arrival_sample": "P",
    "trace_P1_arrival_sample": "P",

    "trace_Pg_arrival_sample": "P",
    "trace_PG_arrival_sample": "P",

    "trace_Pn_arrival_sample": "P",
    "trace_PmP_arrival_sample": "P",
    "trace_pwP_arrival_sample": "P",
    "trace_pwPm_arrival_sample": "P",
    
    "trace_s_arrival_sample": "S",
    "trace_S_arrival_sample": "S",
    "trace_S1_arrival_sample": "S",

    "trace_Sg_arrival_sample": "S",
    "trace_SG_arrival_sample": "S",

    "trace_SmS_arrival_sample": "S",
    "trace_Sn_arrival_sample": "S",
}

In [ ]:
class Tapering:
    def __init__(self, alpha=0.3, key='X'):
        self.alpha = alpha  # Tapering Coefficient
        if isinstance(key, str):
            self.key = (key, key)
        else:
            self.key = key

    def __call__(self, state_dict):
        x, metadata = state_dict[self.key[0]]
        taper = signal.windows.tukey(x.shape[-1], self.alpha)
        x = x * taper
        state_dict[self.key[1]] = (x, metadata)

In [ ]:
sps = 100
augmentations = [
    # Tapering(),
    # sbg.Filter(N=4,
    #            Wn=[1],
    #            btype='highpass',
    #            forward_backward=True,
    #            ),
    sbg.Normalize(
        demean_axis=-1,
        amp_norm_axis=-1,
        amp_norm_type="peak"),
    # sbg.FixedWindow(
    #     p0=-15*sps,
    #     windowlen=1*60*sps,
    #     strategy="pad",
    #     key='X'),
    # sbg.WindowAroundSample(
    #     metadata_keys=list(phase_dict.keys()),
    #     samples_before=2000,
    #     windowlen=5000,
    #     selection="random",
    #     strategy="variable"),
    # sbg.GaussianNoise(
    #     scale=(0, 0.02),
    #     key='X'),
    # sbg.RandomWindow(
    #     windowlen=3001),
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(
        label_columns=phase_dict,
        model_labels=cfg.training.hyperparameters.phases,
        sigma=30,
        dim=0),
]

In [ ]:
generator = sbg.GenericGenerator(dataset)
generator.add_augmentations(augmentations)

In [ ]:
n = [18, 30,37,76,87,97,105,109,126,137,149,158,161,173,179,196,204,209, 217,
     222, 230,235,251,254,257,261,260,265,266,267,268,273,275,290,302,310,
     312,323,331,335,355,367,376,388,391,393]

In [ ]:
for ii in range(len(dataset.metadata)):
    sample = dataset.get_sample(ii)
    _data, _meta = sample
    condition, quality = srw.health_check.routine.is_waveform_healthy(_data, axis=1, max_thr=1e-6, std_thr=1e-5)
    if not all(condition):
        # print(ii, quality)
        dataset.metadata.loc[_meta['index'], 'split'] = 'Undefined'
    else:
        snr_thr = 3
        snr = srw.health_check.routine.compute_snr(trace=_data, pick_idx=500, noise_window=100, signal_window=200)
        if (snr < snr_thr).all():
            # print(ii, snr)
            dataset.metadata.loc[_meta['index'], 'split'] = 'Undefined'

In [ ]:
srw.health_check.routine.is_waveform_healthy(sample["X"], axis=1, max_thr=1e-6, std_thr=1e-5)

In [ ]:
# # Sample data (replace with your actual signal)
# t = np.linspace(0, 1, 1000, False)  # 1 second
# carrier_frequency = 50  # Hz
# modulating_frequency = 5  # Hz
# signal = np.cos(2 * np.pi * carrier_frequency * t) * (0.5 + 0.5 * np.cos(2 * np.pi * modulating_frequency * t))

# # Calculate analytic signal
# analytic_signal = scsig.hilbert(signal)

# # Calculate envelope
# envelope = np.abs(analytic_signal)

# # Plot the results
# plt.plot(t, signal, label='Original Signal')
# plt.plot(t, envelope, label='Envelope', linewidth=2)
# plt.plot(t, -envelope,  linewidth=2) # Plot negative envelope for visualization
# plt.xlabel("Time")
# plt.ylabel("Amplitude")
# plt.title("Signal and its Envelope")
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
sample = generator[439]
x = sample["X"]
plt.plot(x[0].T)

In [ ]:
data = np.sum(x**2, axis=0)
data = x[2, :] ** 2
npts = x.size
samprate = 100
# Calculate analytic signal
box = np.ones(5)
data_smoothed = np.convolve(data, box)
peaks, _ = find_peaks(data_smoothed)

analytic_signal = scsig.hilbert(data)

# Calculate envelope
envelope = np.abs(analytic_signal)

plt.plot(data, label='Envelope', linewidth=2)

plt.plot(peaks, data[peaks], label='peaks', linewidth=1)
plt.legend()

In [ ]:
np.correlate(envelope, sample['y'][1:].sum(axis=0))

In [ ]:
correlate(a=envelope, b=sample['y'][1:].sum(axis=0), shift=0, demean=True, normalize='naive', method='auto')

In [ ]:
model = sbm.PhaseNet().from_pretrained('original')
sample = generator[37]
x = sample["X"]
y = sample['y']

data = torch.tensor(x, device=model.device).unsqueeze(0)
pred = model(data)


In [ ]:
a = np.random.randn(3, 3001)
b = np.random.randn(3, 3001)

In [ ]:
a_m = a - np.mean(a, axis=1, keepdims=True)
b_m = b - np.mean(b, axis=1, keepdims=True)

cov = np.sum(a_m * b_m, axis=1)
std = np.std(a, axis=1) * np.std(b, axis=1) * a.shape[1]

row_corr_vec = cov / std
print("Row-wise (vectorized):", row_corr_vec)

In [ ]:

# Example arrays (replace with your data)
# a = b = np.random.randn(3, 3001)
# b = np.random.randn(3, 3001)

# Row-wise correlation
row_corr = np.zeros(3)
for i in range(3):
    row_corr[i] = np.corrcoef(a[i, :], b[i, :])[0, 1]

print("Row-wise correlations:", row_corr)

In [ ]:
ii = 67
# ii = n[3]
print(ii, '*'*100)
metadata = dataset.metadata.iloc[[ii]]
sample = generator[ii]
x = sample["X"]
###################################################################
with pd.option_context("display.max_columns", None):
    display(metadata)
###################################################################
fig, axes = plt.subplots(1, 2,
        figsize=(15, 4))
jj = -1
for _x, channel in zip(x, dataset.component_order):
    freq, ampl = fft(array=_x, delta=0.01)
    axes[0].plot(_x+jj, label=channel)
    axes[1].semilogx(freq, ampl, label=channel)
    jj += 1
plt.legend()
plt.show()
###################################################################
skewness_values = [skew(_) for _ in x]
skewness_values = [abs(el) for el in skewness_values]
print(skewness_values)
###################################################################
for _x, channel in zip(x, dataset.component_order):
    cs = np.cumsum(_x)
    print(channel, cs.min(), cs.max())
    plt.plot(cs, label=channel)
plt.legend()
plt.show()


In [ ]:
ii = 126
# ii = n[3]
print(ii, '*'*100)
metadata = dataset.metadata.iloc[[ii]]
sample = generator[ii]
x = sample["X"]
###################################################################
with pd.option_context("display.max_columns", None):
    display(metadata)
###################################################################
fig, axes = plt.subplots(1, 2,
        figsize=(15, 4))
jj = -1
for _x, channel in zip(x, dataset.component_order):
    freq, ampl = fft(array=_x, delta=0.01)
    axes[0].plot(_x[800: 1000]+jj, label=channel)
    axes[1].semilogx(freq, ampl, label=channel)
    jj += 1
plt.legend()
plt.show()
###################################################################
skewness_values = [skew(_) for _ in x]
skewness_values = [abs(el) for el in skewness_values]
print(skewness_values)
###################################################################
for _x, channel in zip(x, dataset.component_order):
    cs = np.cumsum(_x)
    print(channel, cs.min(), cs.max())
    plt.plot(cs, label=channel)
plt.legend()
plt.show()


In [ ]:
from scipy.stats import skew

# Assuming signals is a list or array of the three signals
skewness_values = [skew(_) for _ in x]
skewness_values = [abs(el) for el in skewness_values]
skewness_values

In [ ]:
metadata['trace_N_lower_quartile_counts'], metadata['trace_N_upper_quartile_counts']

In [ ]:
for channel in "ENZ":
    r = abs(metadata[f'trace_{channel}_max_counts'].values) / abs(metadata[f'trace_{channel}_min_counts'].values)
    print(r)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

signal = tmp#sample["X"][0]
# Fit a normal distribution to the data
mu, std = norm.fit(signal)

# Plot histogram
plt.hist(signal, bins=50, density=True, alpha=0.6, color='g')

# Plot probability density function
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
tmp = abs(x[0])
tmp = tmp - np.mean(tmp)
print(list(tmp)[400:2000])
print(list(x[0])[400:2000])
plt.plot(tmp); plt.show()

for data in [tmp, x[0]]:
    lim = max(abs(data.min()), abs(data.max()))
    step = lim/20
    plt.hist(data, bins=np.arange(-lim, lim+step, step)-(step/2))
    plt.show()

In [ ]:
model_phasenet = sbm.PhaseNet().from_pretrained('original')
ii = n[1]
for ii in n:
    print(ii, '*'*100)
    sample = generator[ii]
    x = sample["X"]
    result = {}
    fig, axes = plt.subplots(1, 2,
            figsize=(15, 4))
    jj = -1
    skewness_values = [skew(_) for _ in x]
    skewness_values = [abs(el) for el in skewness_values]
    print('skewness', skewness_values)
    for _x, channel in zip(x, dataset.component_order):
        freq, ampl = fft(array=_x, delta=0.01)
        axes[0].plot(_x+jj, label=channel)
        axes[1].semilogx(freq, ampl, label=channel)
        ####
        fft_low = ampl[freq<1]
        fft_mid = ampl[(1<=freq) & (freq<20)]
        fft_hig = ampl[20<=freq]
        d = {f'trace_{channel}_fft_max_lt1hz': fft_low.max().round(3),
            f'trace_{channel}_fft_max_ge1hz_lt20hz': fft_mid.max().round(3),
            f'trace_{channel}_fft_max_gt20hz': fft_hig.max().round(3),
            ###
            f'trace_{channel}_fft_mean_lt1hz': fft_low.mean().round(3),
            f'trace_{channel}_fft_mean_ge1hz_lt20hz': fft_mid.mean().round(3),
            f'trace_{channel}_fft_mean_gt20hz': fft_hig.mean().round(3)}
        ######################################################################
        mode = 'max'
        lst = [f'trace_{channel}_fft_{mode}_lt1hz',
                f'trace_{channel}_fft_{mode}_ge1hz_lt20hz',
                f'trace_{channel}_fft_{mode}_gt20hz']
        _result = [d[key] for key in lst]
        condition = {'calibration_signal': False,
                    'High frequency Noise': False}
        if not _result[0] <  _result[1]:
                condition['calibration_signal'] = True
        if  _result[1] <=  _result[2]*1.1:
            condition['High frequency Noise'] = True
        result.update(d)
        # plt.semilogx(freq, ampl)
        jj += 1
        print(channel, condition)
    plt.legend()
    plt.show()
    x = sample["X"]
    # visual_eval(data_X=sample["X"], data_Y=sample["y"], models={'PhaseNet': model_phasenet}, phases='NPS', title=None)
    # plt.show()
    with pd.option_context("display.max_columns", None):
        display(dataset.metadata.iloc[[ii]])

In [ ]:
with pd.option_context("display.max_columns", None):
    display(dataset.metadata.iloc[[ii]])

In [ ]:
channel = 'Z'
mode = 'max'
result_final = {}
for channel in "NEZ":
    for mode in ['max', 'mean']:
        lst = [f'trace_{channel}_fft_{mode}_lt1hz',
               f'trace_{channel}_fft_{mode}_ge1hz_lt20hz',
               f'trace_{channel}_fft_{mode}_gt20hz']
        _result = [result[key] for key in lst]
        condition = {'calibration_signal': False,
                    'High frequency Noise': False}
        if not _result[0] <  _result[1]:
            condition['calibration_signal'] = True
        if  _result[1] <=  _result[2]*1.1:
            condition['High frequency Noise'] = True
        print(channel, mode, _result, condition)
        result_final[f'h-noise_{channel}_{mode}']

In [ ]:
result['trace_Z_fft_max_gt20hz'], result['trace_Z_fft_max_ge1hz_lt20hz'], 

In [ ]:
result['trace_Z_fft_mean_gt20hz'], result['trace_Z_fft_mean_ge1hz_lt20hz'], 

In [ ]:
result['trace_Z_fft_max_gt20hz']

In [ ]:
model_phasenet = sbm.PhaseNet().from_pretrained('original')
for ii in n:
    print(ii, '*'*100)
    sample = generator[ii]
    x = sample["X"]
    for _x in x:
        freq, ampl = fft(array=_x, delta=0.01)
        plt.semilogx(freq, ampl)
    plt.show()
    x = sample["X"]
    visual_eval(data_X=sample["X"], data_Y=sample["y"], models={'PhaseNet': model_phasenet}, phases='NPS', title=None)
    plt.show()

In [ ]:
plt.plot(pred.detach().numpy()[0].T)

In [ ]:
for index in n:
    print(index, '*'*20)
    sample = generator[index]
    x = sample["X"]
    for _x in x:
        freq, ampl = fft(array=_x, delta=0.01)
        plt.plot(freq, ampl)
    plt.show()
    data = np.sum(x**2, axis=0)
    analytic_signal = scsig.hilbert(data)
    # Calculate envelope
    envelope = np.abs(analytic_signal)
    cc = correlate(a=envelope, b=sample['y'][1:].sum(axis=0), shift=0, demean=True, normalize='naive', method='auto')
    
    print(f'{index=} {cc=}')
    
    condition, quality = srw.health_check.routine.is_waveform_healthy(sample["X"], axis=1, max_thr=1e-6, std_thr=1e-5)
    print(condition, quality)
    fig = plt.figure(figsize=(10, 5))
    axs = fig.subplots(2, 1, sharex=True, gridspec_kw={"hspace": 0,
                                                       "height_ratios": [3, 1]})
    axs[0].plot(sample["X"].T + [1, 2, 3])
    axs[1].plot(sample["y"].T)
    plt.suptitle(index)
    plt.show()

In [ ]:
lst_snr = []
lst_cc = []

for index in range(len(generator)):
    print(index)
    sample = generator[index]
    x = sample["X"]

    data = np.sum(x**2, axis=0)
    analytic_signal = scsig.hilbert(data)
    # Calculate envelope
    envelope = np.abs(analytic_signal)
    cc = correlate(a=envelope, b=sample['y'][1:].sum(axis=0), shift=0, demean=True, normalize='naive', method='auto')
    print(f'{index=} {cc=}')
    
    condition, quality = srw.health_check.routine.is_waveform_healthy(sample["X"], axis=1, max_thr=1e-6, std_thr=1e-5)
    snr = srw.health_check.routine.compute_snr(data=sample["X"], pick_idx=500,
                                               noise_window=100, signal_window=200,
                                               domain='time', axis=1, epsilon=1e-8)
    # print('SNR:', snr)
    # print(condition, quality)
    lst_snr.append(snr)
    lst_cc.append(cc)
    
    # fig = plt.figure(figsize=(10, 5))
    # axs = fig.subplots(2, 1, sharex=True, gridspec_kw={"hspace": 0,
    #                                                    "height_ratios": [3, 1]})
    # axs[0].plot(sample["X"].T + [1, 2, 3])
    # axs[1].plot(sample["y"].T)
    # plt.suptitle(index)
    # plt.show()

In [ ]:
snr_new = np.array(lst_snr)
snr_new

In [ ]:
_ = plt.hist(snr_new[:, 0], bins=range(0, 30), log=False)

In [ ]:
_ = plt.hist(snr_new[:, 1], bins=range(0, 30), log=False)

In [ ]:
_ = plt.hist(snr_new[:, 2], bins=range(0, 30), log=False)

In [ ]:
cc_new = np.array(lst_cc)

In [ ]:
plt.hist(cc_new.T[0], bins=np.arange(0, 1, 0.05))

In [ ]:
len(cc_new.T[0]>0.4)

In [ ]:
sum(snr_new.T[0]>2)

In [ ]:
sum((cc_new.T[0]>0.4))

In [ ]:
sum((snr_new.T[0]>2) & (cc_new.T[0]>0.4))

In [ ]:
from obspy import read
st = read()
st.filter('bandpass', freqmin=1, freqmax=2)
st.filter('bandpass', freqmin=1, freqmax=1.5)
st.
st[0].stats
# st.write('tmp.mseed', format='MSEED')
# st_new = read('tmp.mseed')
# st_new[0].stats